In [1]:
from topologicpy.Vertex import Vertex
from topologicpy.Face import Face
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Color import Color
from topologicpy.Plotly import Plotly
from topologicpy.Graph import Graph
from topologicpy.Vector import Vector

from topologicpy.Helper import Helper

In [2]:
def getSelectors(cell, keys, values):
    if not cell:
        return None
    # selectors = []
    s = Topology.InternalVertex(cell)
    d = Dictionary.ByKeysValues(keys, values)
    s = Topology.SetDictionary(s, d)
    # selectors.append(s)
    return s

def createPrism(dimension, location):
    w, l, h = dimension
    x, y, z = location
    prism = Cell.Prism(width=w, length=l, height=h, placement="lowerleft")
    prism = Topology.Translate(prism, x, y, z)
    return prism

def createAperture(room1, room2, is_full=True):
    names = [room1, room2]
    names_sel = []
    for sel in selectors:
        if sel.GetDictionary().ValueAtKey('title').StringValue() == room1:
            names_sel.append(sel)
    for sel in selectors:
        if sel.GetDictionary().ValueAtKey('title').StringValue() == room2:
            names_sel.append(sel)
    names_sel

    if not names_sel:
        return

    sub1 = Topology.SelectSubTopology(house, names_sel[0], subTopologyType="cell")
    sub2 = Topology.SelectSubTopology(house, names_sel[1], subTopologyType="cell")
    # print(f'{room1} has {len(Cell.Faces(sub1))} faces   |   {room2} has {len(Cell.Faces(sub2))} faces')
    shared = Topology.SharedFaces(sub1, sub2)
    aperture = Topology.Copy(shared[0])
    aperture = shared[0]
    # print(aperture)

    if is_full:
        return aperture
    else:
        bottom_edge = Face.Edges(aperture)[0]
        for edge in Face.Edges(aperture):
            if Topology.Centroid(edge).Z() < Topology.Centroid(bottom_edge).Z():
                bottom_edge = edge
            
        return Topology.Scale(aperture, x=0.5, z=0.8, origin=Topology.Centroid(bottom_edge))

def getCellFloorArea(cell):
    bottom_face = Cell.Faces(cell)[0]
    for face in Topology.SubTopologies(cell, subTopologyType="face"):
        if Topology.Centroid(face).Z() < Topology.Centroid(bottom_face).Z():
            bottom_face = face
        return Face.Area(bottom_face)

In [34]:
class Space():
    def __init__(self, name,label, dimension, location):
        self.name = name
        self.label = label
        self.dim = dimension
        self.w = dimension[0]
        self.l = dimension[1]
        self.h = dimension[2]
        self.loc = location
        self.x = location[0]
        self.y = location[1]
        self.z = location[2]
        self.topo = createPrism(dimension, location)
        self.selector = getSelectors(self.topo, ["label", "title", "area", "volume"], [self.label, self.name, getCellFloorArea(self.topo), Cell.Volume(self.topo)])
        

In [49]:
houses = []

In [52]:
general_width = 6.5
general_height = 3.0
corridor_width = 1.0

has_patio = True
has_bedroom1 = True
has_bedroom2 = True



In [53]:
selectors = []
cells = []
apertures = []
keys = ["label", "title", "area", "volume"]

parking = Space(
    "Parking",
    0,
    [general_width, 5.0, general_height],
    [0, 0, 0]
)

living = Space(
    "Living Room",
    1,
    [general_width, 4.5, general_height],
    [0,parking.l,0]
)

house = Topology.Boolean(parking.topo, living.topo, operation="merge")

stairs = Space(
    "Stairs",
    2,
    [1.5, 5.0, general_height*2],
    [0,(living.y + living.l),0]
)



cells.extend([
    parking.topo,
    living.topo,
    stairs.topo,  
])

selectors.extend([
    parking.selector,
    living.selector,
    stairs.selector,
])

if has_patio:
    corridor1 = Space(
        "Corridor 1",
        3,
        [corridor_width, stairs.l, general_height],
        [stairs.w,(living.y + living.l),0]
    )
    
    wc1 = Space(
        "WC 1",
        4,
        [stairs.w, 1.2, general_height],
        [0, (stairs.y + stairs.l - 1.2),0]
    )

    wc2 = Space(
        "WC 2",
        4,
        [stairs.w, 1.2, general_height],
        [0, (wc1.y + wc1.l),0]
    )

    service = Space(
        "Service",
        5,
        [stairs.w, 3.0, general_height],
        [0, (wc2.y + wc2.l),0]
    )

    patio = Space(
        "Patio",
        6,
        [(general_width - stairs.w - corridor1.w), stairs.l, general_height*2],
        [(corridor1.x + corridor1.w), (living.y + living.l),0]
    )
    
    kitchen = Space(
        "Kitchen",
        7,
        [1.8, (wc2.l + service.l), general_height],
        [(general_width - 1.8), (patio.y + patio.l),0]
    )

    dining = Space(
        "Dining Room",
        8,
        [(general_width - wc1.w - kitchen.w), kitchen.l, general_height],
        [(wc1.x + wc1.w), (patio.y + patio.l),0]
    )

    backyard = Space(
        "Backyard",
        9,
        [general_width, 3.6, general_height],
        [0, (kitchen.y + kitchen.l),0]
    )    

    cells.extend([
        corridor1.topo,
        wc1.topo,
        wc2.topo,
        service.topo,
        patio.topo,
        kitchen.topo,
        dining.topo,
        backyard.topo,
    ])

    selectors.extend([
        corridor1.selector,
        wc1.selector,
        wc2.selector,
        service.selector,
        patio.selector,
        kitchen.selector,
        dining.selector,
        backyard.selector
    ])

else:
    wc1 = Space(
        "WC 1",
        4,
        [stairs.w, 1.2, general_height],
        [0, (stairs.y + stairs.l - 1.2),0]
    )

    service = Space(
        "Service",
        5,
        [stairs.w, 3.0, general_height],
        [0, (wc1.y + wc1.l),0]
    )
    
    kitchen = Space(
        "Kitchen",
        7,
        [1.8, stairs.l, general_height],
        [(general_width - 1.8), (living.y + living.l),0]
    )

    dining = Space(
        "Dining Room",
        8,
        [(general_width - wc1.w - kitchen.w), kitchen.l, general_height],
        [(wc1.x + wc1.w), (living.y + living.l),0]
    )

    backyard = Space(
        "Backyard",
        9,
        [general_width, service.l, general_height],
        [0, (kitchen.y + kitchen.l),0]
    )    

    cells.extend([
        wc1.topo,
        service.topo,
        kitchen.topo,
        dining.topo,
        backyard.topo,
    ])

    selectors.extend([
        wc1.selector,
        service.selector,
        kitchen.selector,
        dining.selector,
        backyard.selector
    ])

corridor2 = Space(
    "Corridor 2",
    3,
    [corridor_width, stairs.l, general_height],
    [stairs.w, (living.y + living.l), general_height]
)

cells.extend([corridor2.topo])
selectors.extend([corridor2.selector])

if has_bedroom1:
    wc3 = Space(
        "WC 3",
        4,
        [stairs.w, living.l, general_height],
        [0, (parking.y + parking.l), general_height]
    )
    
    bedroom1 = Space(
        "Bedroom 1",
        10,
        [(general_width - wc3.w), living.l, general_height],
        [wc3.w, (parking.y + parking.l), general_height]
    )

    cells.extend([
        wc3.topo,
        bedroom1.topo,
    ])

    selectors.extend([
        wc3.selector,
        bedroom1.selector,
    ])
    
if has_bedroom2:
    if has_patio:
        wc4 = Space(
            "WC 4",
            4,
            [stairs.w, stairs.l, general_height],
            [0, (stairs.y + stairs.l), general_height]
        )
        
        bedroom2 = Space(
            "Bedroom 2",
            10,
            [general_width, dining.l, general_height],
            [wc4.w, (patio.y + patio.l), general_height]
        )

        cells.extend([
            wc4.topo,
            bedroom2.topo,
        ])
    
        selectors.extend([
            wc4.selector,
            bedroom2.selector,
        ])

    else:
        bedroom2 = Space(
            "Bedroom 2",
            10,
            [(general_width - stairs.w - corridor2.w), dining.l, general_height],
            [(corridor2.x + corridor2.w), (living.y + living.l), general_height]
        )

        cells.extend([
            bedroom2.topo,
        ])
    
        selectors.extend([
            bedroom2.selector,
        ])
        
    

for cell in cells:
    house = Topology.Boolean(house, cell, operation="merge")

<topologic_core.Cell object at 0x7f18cbb4f230> <topologic_core.Cell object at 0x7f18ce1877b0>


In [54]:
aperture = createAperture("Parking", "Living Room", False)
apertures.append(aperture)

aperture = createAperture("Living Room", "Stairs", True)
apertures.append(aperture)

if has_patio:
    aperture = createAperture("Living Room", "Corridor 1", True)
    apertures.append(aperture)
    
    aperture = createAperture("Living Room", "Patio", False)
    apertures.append(aperture)

    aperture = createAperture("Patio", "Dining Room", False)
    apertures.append(aperture)

    aperture = createAperture("Patio", "Kitchen", False)
    apertures.append(aperture)

    aperture = createAperture("Dining Room", "Kitchen", True)
    apertures.append(aperture)

    aperture = createAperture("Corridor 1", "WC 1", False)
    apertures.append(aperture)
    
    aperture = createAperture("Corridor 1", "Dining Room", True)
    apertures.append(aperture)

    aperture = createAperture("WC 2", "Service", False)
    apertures.append(aperture)

    aperture = createAperture("Service", "Backyard", False)
    apertures.append(aperture)

    aperture = createAperture("Kitchen", "Backyard", False)
    apertures.append(aperture)

    aperture = createAperture("Dining Room", "Backyard", False)
    apertures.append(aperture)

else:
    aperture = createAperture("Living Room", "Dining Room", False)
    apertures.append(aperture)

    aperture = createAperture("Living Room", "Kitchen", False)
    apertures.append(aperture)

    aperture = createAperture("Dining Room", "Kitchen", True)
    apertures.append(aperture)

    aperture = createAperture("Dining Room", "WC 1", False)
    apertures.append(aperture)

    aperture = createAperture("Service", "Backyard", False)
    apertures.append(aperture)

    aperture = createAperture("Kitchen", "Backyard", False)
    apertures.append(aperture)

    aperture = createAperture("Dining Room", "Backyard", False)
    apertures.append(aperture)

aperture = createAperture("Stairs", "Corridor 2", False)
apertures.append(aperture)

if has_bedroom1:
    aperture = createAperture("Corridor 2", "Bedroom 1", False)
    apertures.append(aperture)

    aperture = createAperture("Bedroom 1", "WC 3", False)
    apertures.append(aperture)

if has_bedroom2:
    aperture = createAperture("Corridor 2", "Bedroom 2", False)
    apertures.append(aperture)
    if has_patio:        
        aperture = createAperture("Bedroom 2", "WC 4", False)
        apertures.append(aperture)
        




In [55]:
selectors = Helper.Flatten(selectors)
cells = Helper.Flatten(cells)
house = CellComplex.ByCells(cells)
house = Topology.TransferDictionariesBySelectors(house, selectors, tranCells=True)
house = Topology.AddApertures(house, apertures, subTopologyType="face", exclusive=True)


In [56]:
houses.append(house)
houses

In [57]:
g = Graph.ByTopology(houses, direct=False, directApertures=True)
graph_data = Plotly.DataByGraph(g, vertexSize=4, vertexGroupKey="label", vertexLabelKey="title", vertexGroups=[0,1,2,3,4,5,6,7,8,9])
bldg_data = Plotly.DataByTopology(house, faceOpacity=0.5, edgeColor="lightgrey", vertexColor="lightgrey")
# bldg_data = []
# graph_data = []
fig = Plotly.FigureByData(graph_data+bldg_data)
Plotly.Show(fig, renderer='jupyterlab')

Graph.ByTopology - Error: The input topology is not a valid topology. Returning None.


TypeError: unsupported operand type(s) for +: 'NoneType' and 'list'

In [58]:
graphs

In [48]:
Graph.ExportToCSV(graphs, "./data", 0, overwrite=True, nodeFeaturesKeys=['area', 'volume'], nodeFeaturesHeader=['area', 'volume'])

Graph.ExportToCSV - Error: The input graph parameter is not a valid topologic graph. Returning None.


In [7]:
title = ["Backyard", "Service"]
selected_cell = []
for cell in Topology.SubTopologies(house, subTopologyType="cell"):
    if cell.GetDictionary().ValueAtKey("title").StringValue() in title:
        selected_cell.append(cell)
print(selected_cell)

AttributeError: 'NoneType' object has no attribute 'StringValue'

In [38]:
# Topology.Show(selected_cell, renderer='jupyterlab', showVertices=True, faceColor="#FFFFFF", backgroundColor="#777777")